In [1]:
!wget "https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip" --no-verbose
!unzip spatio_merged_data_iss.zip
!rm spatio_merged_data_iss.zip

2021-01-18 15:25:42 URL:https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip [37282886/37282886] -> "spatio_merged_data_iss.zip" [1]
Archive:  spatio_merged_data_iss.zip
   creating: input/
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/grid_ML.geojson   
  inflating: __MACOSX/input/._grid_ML.geojson  
  inflating: input/internet_ML.csv   
  inflating: __MACOSX/input/._internet_ML.csv  
  inflating: input/satelite.png      
  inflating: __MACOSX/input/._satelite.png  
  inflating: input/weather.csv       
  inflating: __MACOSX/input/._weather.csv  
  inflating: input/social_pulse_ML.csv  
  inflating: __MACOSX/input/._social_pulse_ML.csv  


In [6]:
!rm -rf functions
!git clone https://github.com/markuslaubenthal/lab_st.git functions

Cloning into 'functions'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 104 (delta 45), reused 76 (delta 24), pack-reused 0
Receiving objects: 100% (104/104), 149.41 KiB | 4.04 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, activations
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

In [9]:
from functions.preprocessing.DataImport import load_and_scale_internet, load_and_scale_satelite, load_and_scale_social, load_and_scale_weather, create_space_invariant
from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file

In [11]:
f = getFileHandler("training_data.h5")

In [13]:
internet, internet_origin, internet_min, internet_max = load_and_scale_internet('input/internet_ML.csv', f)
satelite = load_and_scale_satelite('input/satelite.png', f)
social = load_and_scale_social('input/social_pulse_ML.csv', f)
weather = load_and_scale_weather('input/weather.csv', f)
hour, weekday, holiday = create_space_invariant(f)

In [14]:
x_closeness = generate_dataset(internet, [1,2,3], 168, f, "closeness")
x_period = generate_dataset(internet, [7,14,21,168], 168, f, "period")
y = generate_label(internet, 168, f)

In [15]:
from functions.preprocessing.TestTrainSplit import seven_days_train_test_split

x_closeness_train, x_closeness_test = seven_days_train_test_split(x_closeness, 168)
x_period_train, x_period_test = seven_days_train_test_split(x_period, 168)
y_train, y_test = seven_days_train_test_split(y, 168)

In [ ]:
from functions.model.SplitDenseNetFactory import SplitDenseNetFactory
from keras.optimizers import SGD, Adam
dn = SplitDenseNetFactory()
model = dn.Model(4,3)
lr = 0.01
epochs = 2
model.compile(optimizer=Adam(lr=lr, decay= lr/float(epochs)),
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()]
              )
model.fit([x_closeness_train, x_period_train], y_train, epochs=32)



In [ ]:
# something like
# pred = model.predict([x_closeness,x_period])

In [ ]:
# takes prediction pred for the entire sequence
pred = pred.reshape((pred.shape[0],10000)).T
pred = pred * (internet_max - internet_min)[:,np.newaxis] + internet_min[:,np.newaxis]
pred = np.power(np.full(pred.shape, 10), pred)
base = internet_origin[:,168:]
print('all: ', np.sqrt(((pred-base)**2).mean()))
print('test: ', np.sqrt(((pred[:,:-168]-base[:,:-168])**2).mean()))
print('val: ', np.sqrt(((pred[:,-168:]-base[:,-168:])**2).mean()))

In [ ]:
# reference plot
plt.figure(figsize=(20,10))
plt.plot(range(400), base[2878,100:500], label='base')
plt.plot(range(400), pred[2878,100:500], label='pred')
plt.legend()